In [26]:
## on google colab
from sqlalchemy import create_engine
import pandas as pd
db_location ='sqlite_db_pythonsqlite.db'
engine = create_engine(f'sqlite://///content/sqlite_db_pythonsqlite.db')

con = engine.connect()

## Connection is established

In [21]:
res = con.execute('select * from members limit 5')
for row in res:
  print(row)

(0, 'GUEST', 'GUEST', 'GUEST', 0, '(000) 000-0000', '', '2012-07-01 00:00:00')
(1, 'Smith', 'Darren', '8 Bloomsbury Close, Boston', 4321, '555-555-5555', '', '2012-07-02 12:02:05')
(2, 'Smith', 'Tracy', '8 Bloomsbury Close, New York', 4321, '555-555-5555', '', '2012-07-02 12:08:23')
(3, 'Rownam', 'Tim', '23 Highway Way, Boston', 23423, '(844) 693-0723', '', '2012-07-03 09:32:15')
(4, 'Joplette', 'Janice', '20 Crossing Road, New York', 234, '(833) 942-4710', '1', '2012-07-03 10:25:05')


Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

QUESTIONS:

/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */


/* Q12: Find the facilities with their usage by member, but not guests */


/* Q13: Find the facilities usage by month, but not guests */

In [49]:
"""
    Q10: Produce a list of facilities with a total revenue less than 1000.
    The output of facility name and total revenue, sorted by revenue. Remember
    that there's a different cost for guests and members!
"""
# I think there is a typo in the question.
# there are no facilities with sub $1,000 revenue but several with sub $10,000 revenue
query_compute_total_revenue ="""
          SELECT f.name as Facility_name, 
                  ((count(m.memid = 0) * f.guestcost) + (count(m.memid != 0) * f.membercost)) as total_revenue

            FROM `Members` as m
            join `Bookings` as b on b.memid = m.memid
            join `Facilities` as f on b.facid = f.facid

            group by f.name
			      HAVING total_revenue < 10000 
            order by total_revenue desc
            """

pd.read_sql(query_compute_total_revenue,con)


,Facility_name,total_revenue
0,Massage Room 2,9978.9
1,Squash Court,9240.0
2,Badminton Court,5936.5
3,Pool Table,4180.0
4,Snooker Table,2220.0
5,Table Tennis,2015.0


In [57]:
#/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

get_recommendeding_members ="""
                        select 
                        concat(firstname, ' ', surname) as member_name,
                        group_concat(recommendedby, ',') as rec_ids
                        from Members 
                        group by member_name
                          """
pd.read_sql(get_recommendeding_members,con)
## I think that the recommendedby is the memid of a different memeber that reccomended them. 


#You need another subquesty that retuns the name of the member with the corrospoing id. 



OperationalError: ignored

In [51]:
 #Q12: Find the facilities with their usage by member, but not guests
 get_facility_usage_by_member ="""
          SELECT f.name as Facility_name, 
                count(b.bookid) as member_usage

            FROM `Members` as m
            join `Bookings` as b on b.memid = m.memid
            join `Facilities` as f on b.facid = f.facid
            where m.memid !=0
            Group by Facility_name
            
                          """
pd.read_sql(get_facility_usage_by_member,con)

,Facility_name,member_usage
0,Badminton Court,344
1,Massage Room 1,421
2,Massage Room 2,27
3,Pool Table,783
4,Snooker Table,421
5,Squash Court,195
6,Table Tennis,385
7,Tennis Court 1,308
8,Tennis Court 2,276


In [47]:
 #Find the facilities usage by month, but not guests
usage_by_month ="""
          SELECT strftime('%m', b.starttime) as Month, 
                 count(b.bookid) as member_usage 

            FROM `Members` as m
            join `Bookings` as b on b.memid = m.memid
            join `Facilities` as f on b.facid = f.facid

            where m.memid !=0
            Group by Month
            
                          """
pd.read_sql(usage_by_month,con)

,Month,member_usage
0,07,480
1,08,1168
2,09,1512
